In [1]:
import numpy as np
import pandas as pd
import shapely
import geopandas
import fiona
import h3
import warnings
from shapely import geometry
from shapely.geometry import Polygon
warnings.filterwarnings("ignore")
import pyarrow.parquet as parquet

def style_fn(x):
    return {"color":"blue", "weight":3}

In [2]:
# Drop Z dimension of polygons that occurs often in KML
def _to_2d(x, y, z):
    return tuple(filter(None, [x, y]))

def remove_z(data:geopandas.GeoDataFrame):
    df = data.copy()
    list_geo = []

    # delete altitude calling _to_2d function
    for _, poli in df.iterrows():
        new_shape = shapely.ops.transform(_to_2d, poli.geometry)
        list_geo.append(new_shape)
    
    df.geometry = list_geo
    return df

In [ ]:
'''
distritos = geopandas.read_file('peru_level3.geojson')
distritos=distritos[['NAME_1','NAME_2','NAME_3','TYPE_3','geometry']]
distritos=distritos[distritos['NAME_1']!='Callao']
distritos['NAME_2']=distritos['NAME_2'].str.replace( r"([A-Z])", r" \1").str.strip()
distritos['NAME_3']=distritos['NAME_3'].str.replace( r"([A-Z])", r" \1").str.strip()
distritos['NAME_2']=np.where(distritos['NAME_3']=='Yaquerana','Requena',distritos['NAME_2'])


new_distritos=geopandas.read_file('./distritos_poligonos_github/peru-geojson/peru_distrital_simple.geojson')
new_distritos=new_distritos[['NOMBDEP','NOMBPROV','NOMBDIST','geometry']]
new_distritos['TYPE_3']="Distrito"
new_distritos=new_distritos[['NOMBDEP','NOMBPROV','NOMBDIST','TYPE_3','geometry']]
new_distritos.columns=['NAME_1','NAME_2','NAME_3','TYPE_3','geometry']
new_distritos=new_distritos[(new_distritos['NAME_1']=='CALLAO')&(~new_distritos['geometry'].isna())]
new_distritos['NAME_1']=new_distritos['NAME_1'].apply(str.title)
new_distritos['NAME_2']=new_distritos['NAME_2'].apply(str.title)
new_distritos['NAME_3']=new_distritos['NAME_3'].apply(str.title)
new_distritos['TYPE_3']=new_distritos['TYPE_3'].apply(str.title)

fiona.supported_drivers['KML']='rw'

lima=geopandas.read_file('Lima - Neighborhoods.kml',driver='KML')
lima['Name']=lima['Name'].apply(str.title)
lima=lima[lima['Name']=='La Punta']
lima['NAME_1']='Callao'
lima['NAME_2']='Callao'
lima['TYPE_3']='Distrito'
lima=lima[['NAME_1','NAME_2','Name','TYPE_3','geometry']]
lima.columns=['NAME_1','NAME_2','NAME_3','TYPE_3','geometry']

lima=remove_z(lima)

distritos=geopandas.GeoDataFrame(pd.concat([distritos,new_distritos,lima],ignore_index=True).reset_index(drop=True))
del new_distritos,lima

distritos.columns=['Departamento','Provincia','Distrito','Tipo_Poligono','geometry']
distritos.to_file("distritos_peru.geojson", driver="GeoJSON")
'''

In [ ]:
distritos = geopandas.read_file('distritos_peru.geojson')

# Extraer Departamentos

In [ ]:
import folium 
'''
map_test = folium.Map(tiles="cartodbpositron")

folium.GeoJson(
    distritos,
    style_function=style_fn,
    popup=folium.GeoJsonPopup(distritos.drop(columns=["geometry"]).columns.tolist()),
    tooltip=folium.GeoJsonTooltip(['Departamento','Provincia','Distrito']),
).add_to(map_test)

map_test
'''

In [5]:
dic_hex_geo={'geo_5':'hex_8',
             'geo_6':'hex_9',
             'geo_7':'hex_11',
             'geo_8':'hex_13'}

In [6]:
tipo_geohash=7
tipo_hex=int(dic_hex_geo[f'geo_{tipo_geohash}'][4:])

In [5]:
from typing import List, Tuple

def get_bigger_polygon(row):
    x=geopandas.GeoDataFrame(geometry=list(row['geometry']))
    for i in row.index:
        if i!='geometry':
            x[i]=row[i]
    cols = x.columns.tolist()            
    cols = cols[1:]+[cols[0]]
    x=x[cols]
    x['Area']=x.area/10**6
    x=x[x['Area']==x['Area'].max()][cols]       
    return x

def polygon_to_hexagon(row,res):
    #poli = row.geometry
    poly_geojson = row.geometry.__geo_interface__

    # Parse out geometry key from GeoJSON dictionary
    poly_geojson = poly_geojson['features'][0]['geometry']

    # Fill the dictionary with resolution (res) H3 Hexagons that you want
    h3_hexes = h3.polyfill_geojson(poly_geojson, res)
    df = pd.DataFrame(h3_hexes, columns=['h3_id'])
    for i in row.columns:
        if i!='geometry':
            df[i]=row[i].values[0]
    cols = df.columns.tolist()            
    cols = cols[1:]+[cols[0]]
    df=df[cols]
    return df

# Drop Z dimension of polygons that occurs often in KML
def _to_2d(x, y, z):
    return tuple(filter(None, [x, y]))

# Call the last two functions
def neighborhood_to_hexagon(data:geopandas.GeoDataFrame,res:int,remove_z:bool):
    df = data.copy()
    list_geo = []
    list_df = []

    # delete altitude calling _to_2d function
    if remove_z:
        for _, poli in df.iterrows():
            new_shape = shapely.ops.transform(_to_2d, poli.geometry)
            new_shape=poli.geometry
            list_geo.append(new_shape)    
        df.geometry = list_geo
       
    # Call polygon_to_hexagon function
    for _, row in df.iterrows():
        # drop multipolygons
        if row['geometry'].geom_type=='MultiPolygon':
            temp = get_bigger_polygon(row) 
        else:
            temp=geopandas.GeoDataFrame(geometry=[row['geometry']])
            for i in row.index:
                if i!='geometry':
                    temp[i]=row[i]
            cols = temp.columns.tolist()            
            cols = cols[1:]+[cols[0]]
            temp=temp[cols]
        temp = polygon_to_hexagon(temp,res)
        list_df.append(temp)
    df = pd.concat(list_df).reset_index(drop=True)

    return df

In [ ]:
#distritos=neighborhood_to_hexagon(distritos,res=tipo_hex,remove_z=False)
#distritos=distritos.drop_duplicates(subset='h3_id',keep='first').reset_index(drop=True)

In [ ]:
#distritos.to_parquet(f'./maestro_geohash_distritos/distritos_peru_geohash_{tipo_geohash}.parquet',index=False)

In [ ]:
distritos=pd.read_parquet(f'./maestro_geohash_distritos/distritos_peru_geohash_{tipo_geohash}.parquet')

In [ ]:
distritos

In [6]:
import pygeohash as pgh

def hashingit(lat, long): 
      return pgh.encode(latitude=lat,longitude=long,precision=tipo_geohash)

func_has= np.vectorize(hashingit)

def hex_to_coords(val):
    return [(t[1], t[0]) for t in h3.h3_set_to_multi_polygon([val], geo_json=False)[0][0]]

def geohash_to_polygon(geo):
    """
    :param geo: String that represents the geohash.
    :return: Returns a Shapely's Polygon instance that represents the geohash.
    """
    lat_centroid, lng_centroid, lat_offset, lng_offset = pgh.decode_exactly(geo)

    corner_1 = (lat_centroid - lat_offset, lng_centroid - lng_offset)[::-1]
    corner_2 = (lat_centroid - lat_offset, lng_centroid + lng_offset)[::-1]
    corner_3 = (lat_centroid + lat_offset, lng_centroid + lng_offset)[::-1]
    corner_4 = (lat_centroid + lat_offset, lng_centroid - lng_offset)[::-1]

    return geometry.Polygon([corner_1, corner_2, corner_3, corner_4, corner_1])    

In [ ]:
distritos['Latitud'],distritos['Longitud'] = distritos.apply(lambda rec: h3.h3_to_geo(h=rec['h3_id']), axis=1).str
distritos[f'geohash_{tipo_geohash}'] = func_has(distritos['Latitud'],distritos['Longitud'])
distritos=distritos[distritos.columns.to_list()[:-4]+[f'geohash_{tipo_geohash}']]
distritos = distritos.drop_duplicates(keep='last')
distritos = distritos.reset_index(drop=True)

In [ ]:
distritos

In [ ]:
distritos.to_parquet(f'./maestro_geohash_distritos/distritos_peru_geohash_{tipo_geohash}.parquet',index=False)

In [ ]:
distritos['DUPLICADOS']=np.where(distritos.duplicated(subset=f'geohash_{tipo_geohash}',keep=False),'DUPLICADO','NO_DUPLICADO')

In [ ]:
distritos_hex=distritos.copy()
distritos = geopandas.read_file('distritos_peru.geojson')
distritos=distritos.to_crs(crs = {'init': 'epsg:4326'})
duplicados=distritos_hex[distritos_hex['DUPLICADOS']=='DUPLICADO'].reset_index(drop=True)

distritos_hex=pd.DataFrame(distritos_hex[distritos_hex['DUPLICADOS']!='DUPLICADO']\
                        [['Departamento','Provincia','Distrito','Tipo_Poligono',f'geohash_{tipo_geohash}']])

In [ ]:
distritos_hex.to_parquet(f'./maestro_geohash_distritos/distritos_peru_geohash_{tipo_geohash}.parquet',index=False)
duplicados.to_parquet(f'./maestro_geohash_distritos/distritos_peru_geohash_{tipo_geohash}_duplicados.parquet',index=False)
del distritos_hex

In [ ]:
crs = {'init': 'epsg:4326'}
duplicados['geometry'] = duplicados.apply(lambda rec: geohash_to_polygon(rec[f'geohash_{tipo_geohash}']), axis=1)
duplicados = geopandas.GeoDataFrame(duplicados, crs = crs, geometry = duplicados.geometry)
duplicados['Area']=duplicados.area

In [6]:
def remover_duplicados(df_distritos, df_maestro):
    join=geopandas.overlay(df_distritos, df_maestro[['Distrito','geometry']], how='intersection')
    join['Area_Intersectada']=join.area
    join = join[(join['Distrito_1']==join['Distrito_2'])].reset_index(drop=True)
    join['porcentaje_Area']=np.round(join['Area_Intersectada']*100/join['Area'],decimals=2)
    join=pd.DataFrame(join[['Departamento','Provincia','Distrito_1','Tipo_Poligono',f'geohash_{tipo_geohash}','porcentaje_Area']])
    join.columns=['Departamento','Provincia','Distrito','Tipo_Poligono',f'geohash_{tipo_geohash}','porcentaje_Area']
    max_join=join.groupby(by=['geohash_7'])\
                .agg(porcentaje_Area=pd.NamedAgg(column='porcentaje_Area', aggfunc='max'),).reset_index()
    join=join.merge(max_join,how='inner',on=['geohash_7','porcentaje_Area'])
    return join.reset_index(drop=True)       

In [7]:
duplicados=pd.read_parquet(f'./maestro_geohash_distritos/distritos_peru_geohash_{tipo_geohash}_duplicados.parquet')

In [8]:
crs = {'init': 'epsg:4326'}
duplicados['geometry'] = duplicados.apply(lambda rec: geohash_to_polygon(rec[f'geohash_{tipo_geohash}']), axis=1)
duplicados = geopandas.GeoDataFrame(duplicados, crs = crs, geometry = duplicados.geometry)
duplicados['Area']=duplicados.area

In [9]:
distritos = geopandas.read_file('distritos_peru.geojson')
distritos=distritos.to_crs(crs = {'init': 'epsg:4326'})

In [29]:
join=geopandas.overlay(duplicados, distritos[['Distrito','geometry']], how='intersection')

In [30]:
join['Area_Intersectada']=join.area
join = join[(join['Distrito_1']==join['Distrito_2'])].reset_index(drop=True)
join['porcentaje_Area']=np.round(join['Area_Intersectada']*100/join['Area'],decimals=2)
join=pd.DataFrame(join[['Departamento','Provincia','Distrito_1','Tipo_Poligono',f'geohash_{tipo_geohash}','porcentaje_Area']])
join.columns=['Departamento','Provincia','Distrito','Tipo_Poligono',f'geohash_{tipo_geohash}','porcentaje_Area']
max_join=join.groupby(by=['geohash_7'])\
             .agg(porcentaje_Area=pd.NamedAgg(column='porcentaje_Area', aggfunc='max'),).reset_index()
join=join.merge(max_join,how='inner',on=['geohash_7','porcentaje_Area'])

In [31]:
join

,Departamento,Provincia,Distrito,Tipo_Poligono,geohash_7,porcentaje_Area
0,Amazonas,Bagua,Aramango,Distrito,6r0457b,77.35
1,Amazonas,Bagua,Aramango,Distrito,6r023e8,55.96
2,Amazonas,Bagua,Aramango,Distrito,6r04pxr,61.75
3,Amazonas,Bagua,Aramango,Distrito,6r04hdm,51.63
4,Amazonas,Bagua,Aramango,Distrito,6r04m0h,54.86
...,...,...,...,...,...,...
552235,Ucayali,Coronel Portillo,Yarinacocha,Distrito,6qdckjs,81.36
552236,Ucayali,Coronel Portillo,Yarinacocha,Distrito,6qdckzj,92.13
552237,Ucayali,Coronel Portillo,Yarinacocha,Distrito,6qdbhy2,61.54
552238,Ucayali,Coronel Portillo,Yarinacocha,Distrito,6qdcj4v,53.32


In [15]:
duplicados=remover_duplicados(df_distritos=duplicados,df_maestro=distritos)

In [32]:
duplicados.to_parquet(f'./maestro_geohash_distritos/distritos_peru_geohash_{tipo_geohash}_duplicados.parquet',index=False)
#join.to_parquet(f'./maestro_geohash_distritos/distritos_peru_geohash_{tipo_geohash}_duplicados.parquet',index=False)

In [33]:
distritos_hex=pd.read_parquet(f'./maestro_geohash_distritos/distritos_peru_geohash_{tipo_geohash}.parquet')

In [34]:
distritos = pd.concat([distritos_hex,duplicados]).reset_index(drop=True)
#distritos = pd.concat([distritos_hex,join]).reset_index(drop=True)
distritos['porcentaje_Area']=np.where(distritos['porcentaje_Area'].isna(),100,distritos['porcentaje_Area'])
distritos.to_parquet(f'./maestro_geohash_distritos/distritos_peru_geohash_{tipo_geohash}.parquet',index=False)

In [ ]:
crs = {'init': 'epsg:4326'}
distritos['geometry'] = distritos.apply(lambda rec: geohash_to_polygon(rec[f'geohash_{tipo_geohash}']), axis=1)
distritos=geopandas.GeoDataFrame(distritos, crs = crs, geometry = distritos.geometry)

## Cargar datos a Tutela

In [7]:
distritos=pd.read_parquet(f'./maestro_geohash_distritos/distritos_peru_geohash_{tipo_geohash}.parquet')

In [15]:
distritos=distritos[['Departamento', 'Provincia', 'Distrito', 'Tipo_Poligono', 'geohash_7']]

In [3]:
import os
from google.cloud import bigquery
import pandas as pd
import numpy as np

os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = 'my-key.json'
BigQuery_client = bigquery.Client()

In [16]:
# Full path to the new table  project.dataset.table_name
table_id = 'reportdataexternal-entel-pe.EntelPe.Tb_maestro_GH7'

In [17]:
# Set up a job configuration
job_config = bigquery.LoadJobConfig()
# Submit the job
job = BigQuery_client.load_table_from_dataframe(distritos, table_id, job_config=job_config)  
# Wait for the job to complete and then show the job results
job.result()  

LoadJob<project=reportdataexternal-entel-pe, location=US, id=db13ef6f-5da9-4d05-b596-e073d9d8cb4d>

In [18]:
# Read back the properties of the table
table = BigQuery_client.get_table(table_id)  
print("Table:", table.table_id, "has", table.num_rows, "rows and", len(table.schema), "columns")

Table: Tb_maestro_GH7 has 56545976 rows and 5 columns


## Cargar datos a Omnisci